In [ ]:
!pip install textblob
!pip install textsearch
!pip install contractions 
!pip install afinn
import nltk 
nltk.download('all')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 5.4 MB/s 
     |████████████████████████████████| 287 kB 39.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 436 kB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=d90a0e502da918d1178616fbe4801cfc7959e5acc716fbc1ff9f9da9440deab1
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np 
import nltk 
import textblob
import contractions
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

In [ ]:
data = pd.read_csv("drive/MyDrive/movie_reviews.csv.bz2")

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
reviews = np.array(data['review'])
sentiments = np.array(data['sentiment'])

In [ ]:
from bs4 import BeautifulSoup 
import tqdm
import re
import unicodedata

def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  #stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

In [ ]:
strip_html_tags(data['review'][1])

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')

In [ ]:
def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
  return text

In [ ]:
import string
def pre_process_corpus(doc):
  norm_corpus = []
  for text in tqdm.tqdm(doc):
    text = strip_html_tags(text)
    text = remove_accented_chars(text)
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text,re.I)
    #text = text.translate(str.maketrans('','',string.punctuation))
    text = text.lower()
    text = text.strip()
    text = contractions.fix(text)
    tokens = nltk.word_tokenize(text)
    filtered_words = [token for token in tokens if token not in stop_words]
    text = " ".join(filtered_words)
    text = re.sub(' +',' ',text)
    norm_corpus.append(text)
  return norm_corpus

In [ ]:
%%time
normalized_reviews = pre_process_corpus(reviews)

100%|██████████| 50000/50000 [02:57<00:00, 281.62it/s]

CPU times: user 2min 41s, sys: 1.35 s, total: 2min 43s
Wall time: 2min 57s


In [ ]:
#Textblob
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
sample_review_ids = [1500,2500,13000]

In [ ]:
#Without Pre-processing
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
  print('REVIEW:',review)
  print('Actual Sentiment:',sentiment)
  print("Predicted Sentiment:", textblob.TextBlob(review).sentiment.polarity)
  print('-'*100)

REVIEW: I'm not sure that this comment contains an actual spoiler, but I'm playing it safe, so don't read this if you haven't seen the movie.<br /><br />I adore this movie, and so does everyone I work with -- and that is the point. I spent a large part of my working life in cinema, without being an actor. Such people are the _sung_ heroes of this movie: the gaffers, the pullers, the on-air directors, the lighters and writers, the costume people etc etc, and the whole thing is told from their point of view, at least to a great extent. Most actors are nuts and self-absorbed to the point of absurdity, which is what this movie spoofs so well, but you have to have worked with actors to recognize that this movie is real-life drama! Possible spoiler alert: in one great scene, the two leads, both actors, are _discussing_ how to _discuss_ something personal, something entirely 'out-of-script', with another actor, and they start making up lines, rehearsing them, and critiquing each other's perfo

In [ ]:
#Textblob
test_reviews = np.array(normalized_reviews[35000:])
test_sentiments = sentiments[35000:]
sample_review_ids = [1500,2500,13000]

In [ ]:
#Without Pre-processing
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
  print('REVIEW:',review)
  print('Actual Sentiment:',sentiment)
  print("Predicted Sentiment:", textblob.TextBlob(review).sentiment.polarity)
  print('-'*100)

REVIEW: sure comment contains actual spoiler playing safe , read seen movie.i adore movie , everyone work -- point . spent large part working life cinema , without actor . people _sung_ heroes movie : gaffers , pullers , on-air directors , lighters writers , costume people etc etc , whole thing told point view , least great extent . actors nuts self-absorbed point absurdity , movie spoofs well , worked actors recognize movie real-life drama ! possible spoiler alert : one great scene , two leads , actors , _discussing_ _discuss_ something personal , something entirely 'out-of-script ' , another actor , start making lines , rehearsing , critiquing 's performance.since movie appeared , , '91 , become fashionable , especially tv . hardly anyone done well .
Actual Sentiment: positive
Predicted Sentiment: 0.17161172161172158
----------------------------------------------------------------------------------------------------
REVIEW: shower story loyalty , unstoppable advance modern world detr

In [ ]:
#Afinn
from afinn import Afinn
afn = Afinn(emoticons = True)

In [ ]:
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
  print('REVIEW:',review)
  print('Actual Sentiment:',sentiment)
  print("Predicted Sentiment:", afn.score(review))
  print('-'*100)

REVIEW: sure comment contains actual spoiler playing safe , read seen movie.i adore movie , everyone work -- point . spent large part working life cinema , without actor . people _sung_ heroes movie : gaffers , pullers , on-air directors , lighters writers , costume people etc etc , whole thing told point view , least great extent . actors nuts self-absorbed point absurdity , movie spoofs well , worked actors recognize movie real-life drama ! possible spoiler alert : one great scene , two leads , actors , _discussing_ _discuss_ something personal , something entirely 'out-of-script ' , another actor , start making lines , rehearsing , critiquing 's performance.since movie appeared , , '91 , become fashionable , especially tv . hardly anyone done well .
Actual Sentiment: positive
Predicted Sentiment: 9.0
----------------------------------------------------------------------------------------------------
REVIEW: shower story loyalty , unstoppable advance modern world detriment rather tra

In [ ]:
#Vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
def sentiment_analysis_vader(review, threshold = 0.2,verbose = False):
  analyser = SentimentIntensityAnalyzer()
  scores = analyser.polarity_scores(review)
  agg_score = scores['compound']
  final_sentiment = 'positive' if agg_score > threshold else 'negative'
  return scores,final_sentiment

In [ ]:
scores, senti = sentiment_analysis_vader(test_reviews[1500], threshold = 0.2,verbose = False)


In [ ]:
senti,scores

('positive', {'compound': 0.8659, 'neg': 0.078, 'neu': 0.742, 'pos': 0.18})

In [ ]:
test_sentiments[1500]

'positive'

In [ ]:

#Supervised ML based technique
train_reviews = np.array(normalized_reviews[:35000])
train_sentiments = np.array(sentiments[:35000])

test_reviews = np.array(normalized_reviews[35000:])
test_sentiments = np.array(sentiments[35000:])
sample_review_ids = [1500,2500,13010]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(use_idf = True,min_df = 5,ngram_range = (1,2))
tf_matrix_train = tv.fit_transform(train_reviews)
tf_matrix_test = tv.transform(test_reviews)

In [ ]:
test_sentiments

array(['negative', 'positive', 'negative', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'l2',random_state = 46)
lr.fit(tf_matrix_train,train_sentiments)

tf_predictions = lr.predict(tf_matrix_test)

In [ ]:
labels = ['negative','positive']
print(classification_report(test_sentiments,tf_predictions))


              precision    recall  f1-score   support

    negative       0.91      0.89      0.90      7490
    positive       0.89      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



In [ ]:
https://github.com/Laxminarayen/SSN---Batch12/blob/master/Day%20-%202%20-%20Text%20Preprocessings/05.%20Build%20your%20Text%20Pre-processor.ipynb